In [ ]:
def stock_return(cocv):
    if cocv == 'current price':
        return "The current price of {0} is {1}.".format(company1, getdata_current_price())

    if cocv == 'open price':
        return "The open price of {0} is {1}.".format(company1, getdata_open_price())

    if cocv == 'close price':
        return "The close price of {0} is {1}.".format(company1, getdata_close_price())

    if cocv == 'volume':
        return "The volume of {0} is {1}.".format(company1, getdata_volume())

def getdata_current_price():
    global company2
    stock=Stock(company2)
    current_price=stock.get_price()
    #print(current_price)
    return current_price

def getdata_open_price():
    global company2
    stock=Stock(company2)
    openclose=stock.get_open_close()
    #print(openclose)
    open_price=openclose.get('open').get('price')
    #print(openprice)
    return open_price

def getdata_close_price():
    global company2
    stock=Stock(company2)
    openclose=stock.get_open_close()
    #print(openclose)
    close_price=openclose.get('close').get('price')
    #print(closeprice)
    return close_price

def getdata_volume():
    global company2
    stock = Stock(company2)
    volume = Stock(company2).get_volume()
    return volume

In [ ]:
import json
symbol = open("/Users/wangxuemei/Desktop/symbols.json", encoding='utf-8')
global setting
setting = json.load(symbol)

from iexfinance.stocks import Stock
#import iexfinance functions

# Import necessary modules
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# Create a trainer that uses this config
trainer = Trainer(config.load("config_spacy.yml"))

# Load the training data
training_data = load_data('training_data.json')

# Create an interpreter by training the model
interpreter = trainer.train(training_data)

In [ ]:
def send_message(state, message):
    print("USER : {}".format(message))
    new_state, response = respond(state, message)
    print("BOT : {}".format(response))
    return new_state, response

def respond(state, message):
    intent, cocv = entity_extract(message)
    new_state, response = policy_rules(state, intent, cocv)
    return new_state, response

def entity_extract(message):
    # Extract the entities
    global setting
    global company1
    global company2
    global interpreter
    data = interpreter.parse(message)
    intent = data.get('intent').get('name')
    cocv = 'none'
    if intent == None:
        intent = 'none'
    if intent == 'stock_search':
        if data.get('entities')[1].get('entity') == 'company' :
            company1 = data.get('entities')[1].get('value')      #get company name
            for i in setting:
                if company1 in i.get('name').lower():
                    company2 = i.get('symbol')
                    break
            cocv = data.get('entities')[0].get('value')  #open price/close price/volume
    return intent, cocv

In [ ]:
# Define the states
INIT = 0
PENDING = 1
CHOOSED = 2
THANK = 3

# Define the policy rules dictionary
def policy_rules(state, intent, cocv): 
    if state == INIT and intent == "none":
        new_state=INIT
        response="I'm sorry, I cannot understand you."
    
    if state == INIT and intent == "greet":
        new_state=PENDING
        response="Hi, I can search for stock price for you."
    
    if state == INIT and intent == "stock_search":
        new_state=CHOOSED
        response=stock_return(cocv)
    
    if state == INIT and intent == "thankyou":
        new_state=THANK
        response="You are welcome."
    
    if state == PENDING and intent == "none":
        new_state=PENDING
        response="I'm sorry, I cannot understand you."
    
    if state == PENDING and intent == "greet":
        new_state=PENDING
        response="Hi, which stock do you wanna look at?"
    
    if state == PENDING and intent == "stock_search":
        new_state=CHOOSED
        response=stock_return(cocv)
    
    if state == PENDING and intent == "thankyou":
        new_state=PENDING
        response="You are welcome, but which stock do you wanna look at?"
    
    if state == CHOOSED and intent == "none":
        new_state=CHOOSED
        response="I'm sorry, I cannot understand you."
    
    if state == CHOOSED and intent == "greet":
        new_state=CHOOSED
        response="Hi, which stock do you wanna look at?"
    
    if state == CHOOSED and intent == "stock_search":
        new_state=CHOOSED
        print('prc_choosed_ss:',new_state)
        response=stock_return(cocv)
    
    if state == CHOOSED and intent == "thankyou":
        new_state=CHOOSED
        response="You are very welcome"
    
    if state == THANK and intent == "none":
        new_state=INIT
        response="I'm sorry, would you please be more specific?"
    
    if state == THANK and intent == "greet":
        new_state=PENDING
        response="Hello again! I can search for stock price for you."
    
    if state == THANK and intent == "stock_search":
        new_state=CHOOSED
        response=stock_return(cocv)
    
    if state == THANK and intent == "thankyou":
        new_state=THANK
        response="You are very welcome, do you need more help?"
    return new_state, response

from wxpy import *
bot = Bot(cache_path=True)
my_friend = bot.friends().search('Paul George')[0]
my_friend.send('hello, I am a bot.')
# Define send_messages()
global state 
state = INIT
@bot.register()
def chatbot(message):
    global state
    if message.sender == my_friend:
        msg = message.text
        state, response = send_message(state, msg)
        my_friend.send(response)